# Multiclass Semantic Segmentation using CamVid dataset

In [1]:
#Importing libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

import os
import random


## Check system environment


In [2]:
print("Tensorflow version: ",tf.__version__)

print("Is GPU available: {}".format(tf.test.is_gpu_available()))

W0311 17:22:04.333524  9048 deprecation.py:323] From <ipython-input-2-d03a236d2262>:3: is_gpu_available (from tensorflow.python.framework.test_util) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Tensorflow version:  2.1.0
Is GPU available: True


## Data preparation - Import data, clean and creating structured directory


### Function to import and process frames and mask as tensors

In [3]:
def read_to_tensor(fname, output_height=256, output_width=256, normalize_data=False):
    ''' Function to read imags from given image file path, and provide the resized images as tensors
    Inputs:
        fname - image file path
        output_height - required output image height
        output_width  - required output image width
        normalize data - if True,normalize data to be centered around 0 ( 0 mean, range 0 - 1)
    Outputs:
        Processed image tensors
        
    '''
    
    # Read the image as a tensor
    img_strings = tf.io.read_file(fname)
    imgs_decoded = tf.image.decode_img(img_strings)
    
    #Resize the image
    output = tf.image.resize(imgs_decoded,[output_height,output_weight])
    
    #Normalize data
    if normalize_data:
        output = output / 255
    
    return output
    

## Image directory and size parameters 

In [5]:
img_dir = "..\data\CamSeq01"

#Required image dimensions
IMG_HEIGHT = 256
IMG_WIDTH = 256

BATCH_SIZE = 32


## Reading frames and masks
. Mask file names end with "_L.png"

In [6]:
def separate_data_from_labels(img_dir):
    '''
    '''
#     regexp_path = str(os.path.join(img_dir,"*"))
#     list_ds = tf.data.Dataset.list_files(regexp_path)    
#     for f in list_ds.take(5):
#         print(f.numpy())

    #Get all the file names list from the directory
    all_file_list = [f for f in os.listdir(img_dir) if f.endswith(".png")]
    print("Total PNG files in the folder: {}".format(len(all_file_list)))
    
    # Get actual images
    img_files_list =[os.path.join(img_dir,f) for f in all_file_list if "_L" not in f]

    image_count = len(img_files_list)
    # Get masks - Get the files with _L in their name
    mask_files_list =[os.path.join(img_dir,f) for f in all_file_list if "_L" in f]
    
    print("Trainable frames in the folder: {}".format(image_count))
    print("Mask images in the folder: {}".format(len(mask_files_list)))

    data_list_ds = tf.data.Dataset.from_tensor_slices(img_files_list)
    mask_list_ds = tf.data.Dataset.from_tensor_slices(mask_files_list)

    
    return data_list_ds, mask_list_ds, image_count

# read_images(img_dir)


In [8]:
# Get the X_Train_full and Y_Train_full from the image folder
data_list_ds,mask_list_ds,image_count = separate_data_from_labels(img_dir)

# print("Printing first 5 dataset files")
# for f in data_list_ds.take(5):
#     print(f.numpy())

# print("Printing first 5 mask files")
# for f in mask_list_ds.take(5):
#     print(f.numpy())

STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)

Total PNG files in the folder: 202
Trainable frames in the folder: 101
Mask images in the folder: 101


In [16]:
def decode_img(img,_string, mask_string):
    # Decode the image string 
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_png(img_string, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)

    # Decode the mask string 
    # convert the compressed string to a 3D uint8 tensor
    mask = tf.image.decode_png(mask_string, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    mask = tf.image.convert_image_dtype(mask, tf.float32)

    # resize the image to the desired size.
    return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT]), tf.image.resize(mask, [IMG_WIDTH, IMG_HEIGHT])


def process_path(file_path):        
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    mask_string = tf.strings.split(file_path,'.png')
    mask_string  =  tf.strings.join([mask_string[:-1], '_L.png'])[0]
#     print(img)
#     print(mask_string)
    
    mask = tf.io.read_file(mask_string)
#     asd = decode_img(img, mask)

    return img, mask

# for f in data_list_ds.take(1):
# #     print(f)
# #     print(type(f.numpy()))
#     mask_string = tf.strings.split(f,'.png')
# #     print(type(mask_string[0]))
    
    
#     out = tf.strings.join([mask_string[:-1], '_L.png'])
# #     print(out[0])
#     img,mask = process_path(f)



In [17]:
# Get data and labels
data_and_mask_ds = data_list_ds.map(process_path,num_parallel_calls=tf.data.experimental.AUTOTUNE)
print(data_and_mask_ds)

# mask_ds = label_list_ds.map(process_path,num_parallel_calls=tf.data.experimental.AUTOTUNE)
# print(mask_ds)


<ParallelMapDataset shapes: ((), ()), types: (tf.string, tf.string)>


In [91]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat forever
    ds = ds.repeat()

    ds = ds.batch(BATCH_SIZE)

    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds

In [ ]:
# train_ds = prepare_for_training(data_ds)

# image_batch, label_batch = next(iter(train_ds))

In [84]:
# Display images and mask

def show_batch(image_batch,mask_batch):
    
    plt.figure(figsize=(10,10))
    for i in range(25):
        ax = plt.subplot(5,5,i+1)
        plt.imshow(image_batch[i])
        plt.axis('off')


image_batch =data_ds.take(25)
len(list(image_batch))
# print()
# print(image_batch)
# show_batch(image_batch,None)

for image in data_ds.take(1):
    print("Image shape: ", image.numpy().shape)
    


Image shape:  (256, 256, 3)
